In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [2]:
mnist_dataset, mnist_info = tfds.load('mnist',with_info = True, as_supervised = True)

mnist_train, mnist_test = mnist_dataset['train'],mnist_dataset['test']

In [8]:
num_validation_samples = 0.1*mnist_info.splits['train'].num_examples

num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_validation_samples

<tf.Tensor: shape=(), dtype=int64, numpy=6000>

In [9]:
num_test_samples = mnist_info.splits['test'].num_examples

num_test_samples = tf.cast(num_test_samples, tf.int64)

num_test_samples

<tf.Tensor: shape=(), dtype=int64, numpy=10000>

In [13]:
def scale(image,label):
    
    image = tf.cast(image,tf.float32)
    image /= 255
    
    return image, label
    
    

In [14]:
scaled_train_validation_data = mnist_dataset['train'].map(scale)

scaled_test_data = mnist_dataset['test'].map(scale)

In [15]:
buffer = 10000

shuffled_train_validation_data = scaled_train_validation_data.shuffle(buffer)

validation_data = shuffled_train_validation_data.take(num_validation_samples)
train_data      = shuffled_train_validation_data.skip(num_validation_samples)

batch = 100

train_data      = train_data.batch(batch)
validation_data = validation_data.batch(num_validation_samples)
test_data       = scaled_test_data.batch(num_test_samples)




# takes next batch (it is the only batch)
# because as_supervized=True, we've got a 2-tuple structure
validation_inputs, validation_targets = next(iter(validation_data))

# Model

Hyperparameters to adjust :

- width ->  hidden_layer
- depth ->  tf.keras.layers.Dense(hidden_layer,activation = 'relu'/'sigmoid'/'tanh'/'softmax')
- learning rate -> custom = tf.keras.optimizers.Adam(learning_rate=0.0001)
  followed by model.compile(optimizer = custom,loss='sparse_categorical_crossentropy,metrics = ['accuracy'])

- epochs
- batch size for train set

In [34]:
hidden_layer = 200
output_size = 10
model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape = (28,28,1)),
                            
                            tf.keras.layers.Dense(hidden_layer,activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer,activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer,activation = 'relu'),
    
                            tf.keras.layers.Dense(output_size,activation='softmax')])

In [35]:
## Optimizer, Loss, Metrics

model.compile(
              optimizer = 'adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
             )

In [36]:
## Train

epochss = 10

model.fit(train_data, epochs=epochss, validation_data=(validation_inputs, validation_targets), verbose =2)

Epoch 1/10
540/540 - 4s - loss: 0.2641 - accuracy: 0.9218 - val_loss: 0.1197 - val_accuracy: 0.9637
Epoch 2/10
540/540 - 4s - loss: 0.1009 - accuracy: 0.9687 - val_loss: 0.0797 - val_accuracy: 0.9747
Epoch 3/10
540/540 - 4s - loss: 0.0702 - accuracy: 0.9785 - val_loss: 0.0606 - val_accuracy: 0.9810
Epoch 4/10
540/540 - 4s - loss: 0.0521 - accuracy: 0.9831 - val_loss: 0.0548 - val_accuracy: 0.9822
Epoch 5/10
540/540 - 4s - loss: 0.0412 - accuracy: 0.9868 - val_loss: 0.0562 - val_accuracy: 0.9850
Epoch 6/10
540/540 - 4s - loss: 0.0367 - accuracy: 0.9886 - val_loss: 0.0416 - val_accuracy: 0.9870
Epoch 7/10
540/540 - 3s - loss: 0.0277 - accuracy: 0.9912 - val_loss: 0.0343 - val_accuracy: 0.9898
Epoch 8/10
540/540 - 3s - loss: 0.0286 - accuracy: 0.9904 - val_loss: 0.0279 - val_accuracy: 0.9928
Epoch 9/10
540/540 - 4s - loss: 0.0195 - accuracy: 0.9936 - val_loss: 0.0260 - val_accuracy: 0.9915
Epoch 10/10
540/540 - 3s - loss: 0.0228 - accuracy: 0.9928 - val_loss: 0.0224 - val_accuracy: 0.9928

# Test

- remember to only run the test once because this is the test case.
- multiple runnings will cause overfitting on test data and make this useless.
- multiple runnings can only be done on the train set when tuning hyperparameters
  for greater accuracy on the validation data.

In [37]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 754ms/step - loss: 0.0873 - accuracy: 0.9775


In [38]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.09. Test accuracy: 97.75%
